# Classic code part

In [1]:
import numpy as np
import random

In [2]:
CAT = "c"
DOG = "d"
MOUSE = "m"
EMPTY = "emp"

field = [[MOUSE, EMPTY],
         [DOG, CAT]]

def showField():
    for row in field:
        print(" ".join(row))

In [3]:
showField()

m emp
d c


In [5]:
class State:
    def __init__(self, field, catP):
        self.field = field
        self.catP = catP #Cat position
    
    def __eq__(self, other):
        return isinstance(other, State) and self.field == other.field and self.catP == other.catP

    def __str__(self):
        return f"State(feld={self.field}, cat_pos={self.catP}" 

# actions:
UP = '00'
DOWN = '01'
LEFT = '10'
RIGHT = '11'

ACTIONS = [UP, DOWN, LEFT, RIGHT]



iniState = State(field=field, catP=[1, 1])

print(iniState.__eq__(5))
newField = iniState.field
print(newField)
print(iniState.catP)

False
[['m', 'emp'], ['d', 'c']]
[1, 1]


In [6]:
# choose action to do
def act(state, action):
    
    def newCatPosition(state, action):
        p = deepcopy(state.catP)
        if action == UP:
            p[0] = max(0, p[0] - 1)
        elif action == DOWN:
            p[0] = min(len(state.field) - 1, p[0]+1)
        elif action == LEFT:
            p[1] = max(0, p[1] - 1)
        elif action == RIGHT:
            p[1] = min(len(state.field) - 1, p[1] + 1)
        else:
            raise ValueError(f"Unkown action {action}")

        return p


    p = newCatPosition(state, action)
    fieldItem = state.field[p[0]][p[1]] #Define who lives in the new cat position 
    newField = deepcopy(state.field)

    if fieldItem == DOG:
        reward = -100
        isDone = True # The dog has killed the cat
        newField[p[0]][p[1]] += CAT
    elif fieldItem == MOUSE:
        reward = 1000
        isDone = True # The cat has eaten the mouse
        newField[p[0]][p[1]] += CAT
    elif fieldItem == EMPTY:
        reward = -1
        isDone = False # Nothing has changed
        old = state.catP
        newField[old[0]][old[1]] = EMPTY
        newField[p[0]][p[1]] = CAT
    elif fieldItem == CAT:
        reward = -1
        isDone = False # Nothing has changed
    else:
        raise ValueError(f"Unknown field item {fieldItem}")
    
    return State(field=newField, catP=p), reward, isDone

# Quantum code part

In [18]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister

def get_var_form(params):
    qr = QuantumRegister(2, name="q")
    cr = ClassicalRegister(2, name='c')
    qc = QuantumCircuit(qr, cr)
    qc.u3(params[0][0], params[0][1], params[0][2], qr[0])
    qc.u3(params[1][0], params[1][1], params[1][2], qr[1])
    qc.cx(1,0)
    qc.u3(params[2][0], params[2][1], params[3][2], qr[0])
    qc.u3(params[3][0], params[3][1], params[3][2], qr[1])
    #qc.measure(qr, cr[0])
    return qc
qc_trial = get_var_form([[0,0,0,0],[0.5,0.5,0.5,0],[0,0,0,0],[0.5,0.5,0.5,0]])
qc_trial.draw()

┌───────────┐   ┌───┐  ┌─────────────┐  
q_0: ───┤ U3(0,0,0) ├───┤ X ├──┤ U3(0,0,0.5) ├──
     ┌──┴───────────┴──┐└─┬─┘┌─┴─────────────┴─┐
q_1: ┤ U3(0.5,0.5,0.5) ├──■──┤ U3(0.5,0.5,0.5) ├
     └─────────────────┘     └─────────────────┘
c: 2/═══════════════════════════════════════════